In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [4]:
z = torch.FloatTensor([1,2,3])

In [5]:
hypothesis = F.softmax(z,dim = 0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [6]:
hypothesis.sum()

tensor(1.)

In [16]:
z = torch.rand(3, 5, requires_grad = True)
hypothesis = F.softmax(z,dim=1)
print(hypothesis)

tensor([[0.1392, 0.2867, 0.1421, 0.2554, 0.1766],
        [0.1584, 0.2665, 0.1580, 0.2387, 0.1784],
        [0.2080, 0.1359, 0.1732, 0.2883, 0.1946]], grad_fn=<SoftmaxBackward>)


In [18]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([1, 3, 1])


In [24]:
y_one_hot = torch.zeros_like(hypothesis)
print(y_one_hot)
print(y.unsqueeze(1))
y_one_hot.scatter_(1,y.unsqueeze(1),1)
print(y.unsqueeze(1))
print(y_one_hot)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[1],
        [3],
        [1]])
tensor([[1],
        [3],
        [1]])
tensor([[0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.]])


In [29]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.5592, grad_fn=<MeanBackward0>)


In [32]:
#Low Level
torch.log(F.softmax(z, dim = 1))

tensor([[-1.9722, -1.2492, -1.9513, -1.3648, -1.7339],
        [-1.8429, -1.3225, -1.8449, -1.4324, -1.7238],
        [-1.5700, -1.9959, -1.7536, -1.2438, -1.6367]], grad_fn=<LogBackward>)

In [33]:
#High Level
F.log_softmax(z, dim = 1)

tensor([[-1.9722, -1.2492, -1.9513, -1.3648, -1.7339],
        [-1.8429, -1.3225, -1.8449, -1.4324, -1.7238],
        [-1.5700, -1.9959, -1.7536, -1.2438, -1.6367]],
       grad_fn=<LogSoftmaxBackward>)

In [34]:
#Low Level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim = 1).mean()

tensor(1.5592, grad_fn=<MeanBackward0>)

In [35]:
#High Level
F.nll_loss(F.log_softmax(z, dim = 1), y)

tensor(1.5592, grad_fn=<NllLossBackward>)

In [36]:
F.cross_entropy(z,y)

tensor(1.5592, grad_fn=<NllLossBackward>)

# Training with Low-Level Cross Entropy Loss

In [43]:
x_train = [[1,2,1,1],
           [2,1,3,2],
           [3,1,3,4],
           [4,1,5,5],
           [1,7,5,5],
           [1,2,5,6],
           [1,6,6,6],
           [1,7,7,7]]
y_train = [2,2,2,1,1,1,0,0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [46]:
W = torch.zeros((4,3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W,b], lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = F.softmax(x_train.matmul(W) + b , dim = 1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1),1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim = 1).mean()
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


# Training with F.cross_entropy

In [47]:
W = torch.zeros((4,3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs +1):
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
        ))
        

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [48]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3)
        
    def forward(self, x):
        return self.linear(x)
model = SoftmaxClassifierModel()

In [49]:
optimizer = optim.SGD(model.parameters(),lr = 0.1)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
        ))
        
        

Epoch    0/1000 Cost: 1.961495
Epoch  100/1000 Cost: 0.644224
Epoch  200/1000 Cost: 0.560511
Epoch  300/1000 Cost: 0.506295
Epoch  400/1000 Cost: 0.462047
Epoch  500/1000 Cost: 0.422630
Epoch  600/1000 Cost: 0.385713
Epoch  700/1000 Cost: 0.349799
Epoch  800/1000 Cost: 0.313748
Epoch  900/1000 Cost: 0.277148
Epoch 1000/1000 Cost: 0.246214
